In [ ]:
###REMEMBER TO CLEAR os.chdir() AND RESTART KERNEL AND CLEAR OUTPUT 

###FOR EXAMPLE: AAPL
#https://investor.apple.com/sec-filings/default.aspx
#10Q and 10K in whichStock directory   

In [ ]:
from urllib.request import Request, urlopen

import os
import requests
import re
import json
import pandas as pd

import matplotlib.pyplot as plt

In [ ]:
#get API key file contents and connect to FMP API
#os.chdir('~')

FMP_API_KEY = ""
BASE_URL = ""

with open('FMP_API.txt', 'r') as file: 
    for line in file: 
        if line.startswith('FMP_API_KEY'): 
            FMP_API_KEY = line.split('"')[1]
        elif line.startswith('BASE_URL'): 
            BASE_URL = line.split('"')[1]

ticker = input("Enter ticker symbol for Buffett DCA Analysis: ")

In [ ]:
# without limit parameter in URL: 
annual_income_stmt = pd.DataFrame((requests.get(BASE_URL + '/api/v3/income-statement/' + ticker + '?apikey=' + FMP_API_KEY)).json())
quarterly_income_stmt = pd.DataFrame((requests.get(BASE_URL + '/api/v3/income-statement/' + ticker + '?period=quarter&apikey=' + FMP_API_KEY)).json())

In [ ]:
annual_income_stmt.head()

In [ ]:
quarterly_income_stmt.head()

In [ ]:
#[INCOME STATEMENT][REVENUE][COST OF GOODS SOLD]
#   calculate net earnings, annually and monthly
#   WB formula:   NET REVENUE = TOTAL REVENUE - COGS
#   FMP formula:  REVENUE - COST OF REVENUE

annual_metrics = pd.DataFrame()
annual_metrics['Ticker'], annual_metrics['Frequency'], annual_metrics['Year'], annual_metrics["Filing Date"], annual_metrics["Net Earnings"] = [ticker, '', annual_income_stmt['calendarYear'], annual_income_stmt["fillingDate"], annual_income_stmt['revenue'] - annual_income_stmt['costOfRevenue']]
annual_metrics['Ticker'] = ticker
annual_metrics['Frequency'] = 'Annually'

#annual_metrics

quarterly_metrics = pd.DataFrame()
quarterly_metrics['Ticker'], quarterly_metrics['Frequency'], quarterly_metrics['Year'], quarterly_metrics['Quarter'], quarterly_metrics["Filing Date"], quarterly_metrics["Net Earnings"] = [ticker, '', quarterly_income_stmt['calendarYear'], quarterly_income_stmt['period'], quarterly_income_stmt['fillingDate'], quarterly_income_stmt['revenue'] - quarterly_income_stmt['costOfRevenue']]
quarterly_metrics['Ticker'] = ticker
quarterly_metrics['Frequency'] = 'Quarterly'

#quarterly_metrics


In [ ]:
#[INCOME STATEMENT][COST OF GOODS SOLD]
#   store COGS value in metrics table, annually and quarterly
annual_metrics['COGS'] = annual_income_stmt['costOfRevenue']

quarterly_metrics['COGS'] = quarterly_income_stmt['costOfRevenue']

In [ ]:
annual_metrics.head()

In [ ]:
quarterly_metrics.head()

In [ ]:
#[INCOME STATEMENT][REVENUE][COST OF GOODS SOLD]
#   calculate gross profit margin, annually and monthly
#   WB formula:   GROSS PROFIT MARGIN = GROSS PROFIT / TOTAL REVENUE
#   FMP formula:  GROSS PROFIT MARGIN = GROSS PROFIT / REVENUE

#1 IN DATAFRAME NAME SINCE FIRST TIME MERGING RAW AND CALCULATED DATA
annual_joined_1 = annual_income_stmt.merge(annual_metrics, how = 'inner', left_on = ['fillingDate', 'calendarYear'], right_on = ['Filing Date', 'Year'])
quarterly_joined_1 = quarterly_income_stmt.merge(quarterly_metrics, how = 'inner', left_on = ['fillingDate', 'calendarYear', 'period'], right_on = ['Filing Date', 'Year', 'Quarter'])

#calculations for GPM 
annual_metrics['Gross Profit Margin'] = (annual_joined_1['Net Earnings'] / annual_joined_1['revenue'])
quarterly_metrics['Gross Profit Margin'] = quarterly_joined_1['Net Earnings'] / quarterly_joined_1['revenue']

In [ ]:
#[INCOME STATEMENT][OPERATING EXPENSES][GROSS PROFIT]
#   calculate operating profit or loss, annually and monthly
#   WB formula: TOTAL OPERATING EXPENSES - GROSS PROFIT
#   FMP formula: OPERATING EXPENSES - GROSS PROFIT

#calculations for operating P/L
annual_metrics['Operating P/L'] = (annual_joined_1['revenue'] - annual_joined_1['operatingExpenses'])
quarterly_metrics['Operating P/L'] = (quarterly_joined_1['revenue'] - quarterly_joined_1['operatingExpenses'])

In [ ]:
#API response URLs with limit parameter in URL
#yearly_income_stmt = pd.DataFrame((requests.get(BASE_URL + '/api/v3/income-statement/' + ticker + '?limit=120&apikey=' + FMP_API_KEY)).json())
#monthly_income_stmt = pd.DataFrame((requests.get(BASE_URL + '/api/v3/income-statement/' + ticker + '?period=quarter&limit=120&apikey=' + FMP_API_KEY)).json())

#without limit parameter in URL: 
#yearly_income_stmt = pd.DataFrame((requests.get(BASE_URL + '/api/v3/income-statement/' + ticker + '?apikey=' + FMP_API_KEY)).json())
#monthly_income_stmt = pd.DataFrame((requests.get(BASE_URL + '/api/v3/income-statement/' + ticker + '?period=quarter&apikey=' + FMP_API_KEY)).json())

#response_from_API = yearly_income.json()
#df = pd.DataFrame(response_from_API)